# 1. 청년 인구 및 이동 분석

## 2) 청년 순이동률 
 전입·전출 분석

In [ ]:
import pandas as pd
import requests
import numpy as np

# pd.set_option("display.max_rows", 1000)

admin = pd.read_csv("../../data/행정구역코드.csv")
admin["행정구역코드"] = admin["행정구역코드"].astype(str)

# 수도권 + 광역시 제외
exclude_prefix = ["11", "31", "21", "22", "23", "24", "25", "26"]
sigun = admin[~admin["행정구역코드"].str[:2].isin(exclude_prefix)].copy()

objL1 = "+".join(sigun["행정구역코드"].tolist()) + "+"


In [ ]:
BASE_URL = "https://kosis.kr/openapi/Param/statisticsParameterData.do"

params = {
    "method": "getList",
    "apiKey": "ZWRiNzEyMzEwNGI5OWQ3NzcxNGM1MDNiOGJkOTQ0Y2M=",
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "itmId": "T001+",
    "objL1": objL1,
    "objL2": "21+22+23+",
    "format": "json",
    "jsonVD": "Y",
    "prdSe": "Y",
    "newEstPrdCnt": "5"
}

response = requests.get(BASE_URL, params=params)
df_raw = pd.DataFrame(response.json())

In [ ]:
df = df_raw[[
    "PRD_DE",   # 연도
    "C1",       # 행정코드
    "C1_NM",    # 지역명
    "C2_NM",    # 지표명 (순이동률 / 순이동)
    "DT"        # 값
]].copy()

df.columns = ["연도", "행정코드", "지역명", "지표", "값"]

df["연도"] = df["연도"].astype(int)
df["행정코드"] = df["행정코드"].astype(str)
df["값"] = pd.to_numeric(df["값"], errors="coerce")

rate_df = df[df["지표"].str.contains("순이동률")].copy()
count_df = df[df["지표"].str.contains("순이동") & ~df["지표"].str.contains("률")].copy()

rate_df.rename(columns={"값": "청년순이동률"}, inplace=True)
count_df.rename(columns={"값": "청년순이동"}, inplace=True)

pivot_rate_df = (
    rate_df.pivot_table(
        index=["행정코드", "지역명"],
        columns="연도",
        values="청년순이동률",
        aggfunc="sum"
    )
    .reset_index()
)

year_cols = [
    col for col in pivot_rate_df.columns
    if isinstance(col, int)
]

# 단순 평균
pivot_rate_df["평균"] = (
    pivot_rate_df[year_cols].mean(axis=1)
)

# 중앙값
pivot_rate_df["중앙값"] = (
    pivot_rate_df[year_cols].median(axis=1)
)

n = len(year_cols)
mid = n // 2

pivot_rate_df["증가율"] = (pivot_rate_df[year_cols[mid:]].mean(axis=1) - pivot_rate_df[year_cols[:mid]].mean(axis=1))

def calc_trend_slope(row, year_cols):
    y = row[year_cols].values.astype(float)
    x = np.arange(len(y))
    
    if np.all(np.isnan(y)):
        return np.nan
    
    slope = np.polyfit(x, y, 1)[0]
    return slope

pivot_rate_df["성장추세_기울기"] = pivot_rate_df.apply(
    calc_trend_slope,
    axis=1,
    year_cols=year_cols
)

vals = pivot_rate_df[year_cols].to_numpy(dtype=float)
diff = np.diff(vals, axis=1)

pivot_rate_df["꾸준증가점수"] = (diff >= 0).mean(axis=1)

vals = pivot_rate_df[year_cols].to_numpy(dtype=float)
diff = np.diff(vals, axis=1)

# 개선(증가)한 부분만 누적
pivot_rate_df["누적증가량"] = np.maximum(diff, 0).sum(axis=1)

# 단순 평군 TOP 5

In [ ]:
display(pivot_rate_df.sort_values("평균", ascending=False).head(10))

# 중앙값 TOP 5

In [ ]:
display(pivot_rate_df.sort_values("중앙값", ascending=False).head(10))

# 증가율 TOP 5

In [ ]:
display(pivot_rate_df.sort_values("증가율", ascending=False).head(10))

# 신안군
# https://www.youtube.com/watch?v=zrwe4b0Dwqo#:~:text=%EC%A0%84%EB%9D%BC%EB%82%A8%EB%8F%84%20%EC%8B%A0%EC%95%88%EA%B5%B0%EC%9D%80%20%EB%8C%80%EB%B6%80%EB%B6%84%EC%9D%98%20%EB%86%8D%EC%96%B4%EC%B4%8C%20%EC%A7%80%EC%97%AD%EC%9D%B4%20%EC%9D%B8%EA%B5%AC%20%EA%B0%90%EC%86%8C,%EC%A3%BC%EB%AF%BC%20%EC%B0%B8%EC%97%AC%ED%98%95%20%EC%86%8C%EB%93%9D%20%EC%82%AC%EC%97%85%EC%9D%B4%20%EC%A3%BC%EC%9A%94%20%EC%9A%94%EC%9D%B8%EC%9C%BC%EB%A1%9C%20%EB%B6%84%EC%84%9D%EB%90%A9%EB%8B%88%EB%8B%A4.

In [ ]:
import koreanize_matplotlib
import matplotlib.pyplot as plt

year_cols = sorted([c for c in pivot_rate_df.columns if isinstance(c, int)])


# 2) 평균 TOP 5 지역 선택
top5 = (
    pivot_rate_df
    .sort_values("증가율", ascending=False)
    .head(10)
    .copy()
)

# 3) long 형태로 변환 (그래프용)
plot_df = top5.melt(
    id_vars=["행정코드", "지역명", "증가율"],
    value_vars=year_cols,
    var_name="연도",
    value_name="청년순이동률"
).sort_values(["지역명", "연도"])

# 4) 선 그래프 (기본 컬러 사이클로 지역별 자동 색상)
plt.figure(figsize=(10, 6))
for region, g in plot_df.groupby("지역명"):
    plt.plot(g["연도"], g["청년순이동률"], marker="o", label=region)

plt.xticks(year_cols)
plt.xlabel("연도")
plt.ylabel("청년순이동률")
plt.title("평균 청년순이동률 TOP 5 지역의 연도별 추이")
plt.grid(True, alpha=0.3)
plt.legend(title="지역명")
plt.tight_layout()
plt.show()

top5[["행정코드", "지역명", "증가율"]]


# 청년순이동률 그래프

In [ ]:
import koreanize_matplotlib
import matplotlib.pyplot as plt

year_cols = sorted([c for c in pivot_rate_df.columns if isinstance(c, int)])


# 2) 평균 TOP 5 지역 선택
top5 = (
    pivot_rate_df
    .sort_values("평균", ascending=False)
    .head(5)
    .copy()
)

# 3) long 형태로 변환 (그래프용)
plot_df = top5.melt(
    id_vars=["행정코드", "지역명", "평균"],
    value_vars=year_cols,
    var_name="연도",
    value_name="청년순이동률"
).sort_values(["지역명", "연도"])

# 4) 선 그래프 (기본 컬러 사이클로 지역별 자동 색상)
plt.figure(figsize=(10, 6))
for region, g in plot_df.groupby("지역명"):
    plt.plot(g["연도"], g["청년순이동률"], marker="o", label=region)

plt.xticks(year_cols)
plt.xlabel("연도")
plt.ylabel("청년순이동률")
plt.title("평균 청년순이동률 TOP 5 지역의 연도별 추이")
plt.grid(True, alpha=0.3)
plt.legend(title="지역명")
plt.tight_layout()
plt.show()

top5[["행정코드", "지역명", "평균"]]


In [ ]:
pivot_count_df = (
    count_df.pivot_table(
        index=["행정코드", "지역명"],
        columns="연도",
        values="청년순이동",
        aggfunc="sum"
    )
    .reset_index()
)

year_cols = [
    col for col in pivot_rate_df.columns
    if isinstance(col, int)
]

# 단순 평균
pivot_count_df["평균"] = (
    pivot_count_df[year_cols].mean(axis=1)
)

# 중앙값
pivot_count_df["중앙값"] = (
    pivot_count_df[year_cols].median(axis=1)
)

# 증가율 정의
pivot_count_df["증가율"] = (pivot_count_df[year_cols[-2:]].mean(axis=1)- pivot_count_df[year_cols[:2]].mean(axis=1))

In [ ]:
display(pivot_count_df.sort_values("평균", ascending=False).head(200))

In [ ]:


# merged = rate_df.merge(
#     count_df[["연도", "행정코드", "청년순이동"]],
#     on=["연도", "행정코드"],
#     how="inner"
# )

# summary = (
#     merged
#     .groupby(["행정코드", "지역명"])
#     .agg(
#         순이동_1년=("청년순이동", lambda x: x.iloc[-1]),
#         순이동_2년=("청년순이동", lambda x: x.iloc[-2:].sum()),
#         순이동_3년=("청년순이동", "sum"),
#         평균_청년순이동률=("청년순이동률", "mean")
#     )
#     .reset_index()
# )


# 청년 유입 비율이 높은 지역 TOP 20

In [ ]:
# top_rate = (
#     summary
#     .sort_values("평균_청년순이동률", ascending=False)
#     .head(10)
# )

# display(top_rate)

In [ ]:
import matplotlib.pyplot as plt

pivot_count_df
test = pivot_count_df.loc[pivot_count_df["지역명"]=="완주군", [2022, 2023, 2024]].iloc[0].tolist()
test


plt.plot()
plt.plot(['2022', '2023', '2024'], test)

plt.legend( ['완주군']
        , fontsize = 12
        , loc = 'upper left'  )
 


In [ ]:
top_count = (
    summary
    .sort_values("순이동_3년", ascending=False)
    .head(10)
)

display(top_count)